In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
import ast 
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import pandas as pd
from pathlib import Path
import tensorflow_io as tfio
import ast
from pathlib import Path
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow import keras 

#Select the Audio File
AUDIOFILE = '345.wav'

file = os.path.join('..', 'Generator', 'audiodata', AUDIOFILE)

def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

def preprocess(file_path): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:63901] 
    zero_padding = tf.zeros([63901] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    
    return spectrogram

spectrogram = preprocess(file)

model = keras.models.load_model(os.path.join('..', 'Train', 'model'))

spectrogram = tf.expand_dims(spectrogram, axis=0)
spectrogram.shape
synthesizer = model.predict(spectrogram)

1/1 [==============================] - 0s 315ms/step


In [2]:
note = ['a','b','c','d','e','f','g']
osc = ['sine','square','sawtooth','triangle']


note = note[synthesizer[0][0].astype(float).argmax()]
osc = osc[synthesizer[1].astype(float).argmax()]
mod = float(synthesizer[2][0])
attack = float(synthesizer[3][0])
decay = float(synthesizer[4][0])
sustain = float(synthesizer[5][0])
release = float(synthesizer[6][0])

In [3]:
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pathlib import Path
from scipy.io.wavfile import write
from scipy.signal import butter, lfilter

sr = 44100

# Create the frequency
def frequency(note):                                        
    if note == 'c':
        freq = 440 * (2 ** ((60 - 69) / 12))
    elif note == 'd':
        freq = 440 * (2 ** ((62 - 69) / 12))
    elif note == 'e':
        freq = 440 * (2 ** ((64 - 69) / 12))
    elif note == 'f':
        freq = 440 * (2 ** ((65 - 69) / 12))
    elif note == 'g':
        freq = 440 * (2 ** ((67 - 69) / 12))
    elif note == 'a':
        freq = 440 * (2 ** ((69 - 69) / 12))
    elif note == 'b':
        freq = 440 * (2 ** ((71 - 69) / 12))
    else:
        raise ValueError("Invalid note")
    return freq

# Create the oscillator
def oscillator(osc, freq, duration):                     
    if osc == 'sine':
        osc = np.sin(2 * np.pi * np.arange(sr * duration) * freq / sr)
    elif osc == 'square':
        osc = np.sign(np.sin(2 * np.pi * np.arange(sr * duration) * freq / sr))
    elif osc == 'sawtooth':
        osc = 2 * (np.arange(sr * duration) * freq / sr - np.floor(np.arange(sr * duration) * freq / sr)) - 1
    elif osc == 'triangle':
        osc = 2 * np.abs(2 * (np.arange(sr * duration) * freq / sr - np.floor(np.arange(sr * duration) * freq / sr)) - 1) - 1
    elif osc == 'noise':
        osc = np.random.normal(0, 1, duration * sr)
    else:
        raise ValueError("Invalid oscillator type")
    return osc

# Create the envelope for the note
def envelope(attack, decay, sustain, release, duration):    
    envelope = np.zeros(int(sr * duration))
    envelope[:int(attack * sr)] = np.linspace(0, 1, int(attack * sr))
    envelope[int(attack * sr):(int(attack * sr) + int(decay * sr))] = np.linspace(1, sustain, int(decay * sr))
    envelope[(int(attack * sr) + int(decay * sr)):(int(attack * sr) + int(decay * sr) + int(1 * sr))] = sustain
    envelope[(int(attack * sr) + int(decay * sr) + int(1 * sr)):(int(attack * sr) + int(decay * sr) + int(1 * sr) + int(release * sr))] = np.linspace(sustain, 0, int(release * sr))
    return envelope

 # Create the LFO    
def lfo(mod, duration):                                         
    lfo = np.sin(2 * np.pi * np.arange(sr * duration) * mod / sr)
    return lfo

# Create the LP-Filter
def lp_filter(sound, cutoff, sr=sr, order=2):               
    nyq = 0.5 * sr
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, sound)
    return y

#Create the sound
def synthesizer(note, osc, mod, attack, decay, sustain, release, cutoff):    
    # Set the maximum duration of the note
    duration = 4

    # Create the frequency
    freq = frequency(note)
    
    # Create the oscillator
    sound = oscillator(osc, freq, duration)

    # Apply the envelope to the oscillators
    sound *= envelope(attack, decay, sustain, release, duration)

    # Apply amplitude modulation
    """ if mod == True:
        sound *= (1 + lfo(duration)) / 2 """
    sound *= (1 + lfo(mod, duration)) / 2

    # Low-Pass Filter
    sound = lp_filter(sound, cutoff, order=4)

    # Normalize the sound
    sound = sound / np.max(np.abs(sound))

    return sound

sound = synthesizer(note,osc,mod,attack,decay,sustain,release,2000)
sf.write("output.wav", sound, sr, 'PCM_16')